# DRO Price Uncertainty
## fixed demand and leadtime

In [4]:
import numpy as np
import pandas as pd
import pprint
import math
from gurobipy import *
import gurobipy as gp
import cvxpy as cp
import mosek
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from oos_analys_DPL import OOS_analys
import warnings
import time
# Ignore all warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import pandas as pd
import warnings
from scipy.special import gamma

########################==============================models for out sample generation==============================########################  
def get_gaussian_demand(time_horizon, mean, std_dev, rho, M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    mean_vector = np.full((time_horizon,), mean)
    covariance_matrix = np.diag(np.full((time_horizon,), std_dev**2))
    samples = np.random.multivariate_normal(mean_vector, covariance_matrix, M)
    
    cov_value = rho * std_dev**2
    covariance_matrix = np.diag(np.full((time_horizon,), std_dev**2))

    for j in range(time_horizon - 1):
        covariance_matrix[j, j+1] = covariance_matrix[j+1, j] = cov_value
    
    samples_with_correlation = np.random.multivariate_normal(mean_vector, covariance_matrix, M)
    samples_with_correlation = np.round(samples_with_correlation).astype(int)
    samples_df = pd.DataFrame(samples_with_correlation.T)
    
    warnings.filterwarnings("ignore", message="covariance is not symmetric positive-semidefinite")

    return samples_df

def get_gamma_demand(time_horizon, mean, std_dev, M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    # Number of random variables to generate
    k = mean**2 / std_dev**2
    theta = std_dev**2 / mean
    m = time_horizon * M
    samples = np.random.gamma(shape=k, scale=theta, size=m)
    samples = np.round(samples).astype(int)
    reshaped_array = samples.reshape((time_horizon, M))
    samples_df = pd.DataFrame(reshaped_array)

    return samples_df

def get_lognormal_demand(time_horizon, mean, std_dev, rho, M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    normal_mean = np.log(mean**2 / np.sqrt(std_dev**2 + mean**2))
    normal_std_dev = np.sqrt(np.log(1 + (std_dev**2 / mean**2)))

    mean_vector = np.full((time_horizon,), normal_mean)
    covariance_matrix = np.diag(np.full((time_horizon,), normal_std_dev**2))
    cov_value = rho * normal_std_dev**2

    for j in range(time_horizon - 1):
        covariance_matrix[j, j+1] = covariance_matrix[j+1, j] = cov_value

    warnings.filterwarnings("ignore", message="covariance is not symmetric positive-semidefinite")
    samples = np.random.multivariate_normal(mean_vector, covariance_matrix, M)

    lognormal_samples = np.exp(samples)
    lognormal_samples = np.round(lognormal_samples).astype(int)
    samples_df = pd.DataFrame(lognormal_samples.T)

    return samples_df


def get_weibull_demand(time_horizon, mean, std_dev, M, seed=None):
    # 计算形状参数 k 和尺度参数 lambda
    # 使用公式：mean = lambda * Gamma(1 + 1/k) 和 std_dev = lambda * sqrt(Gamma(1 + 2/k) - Gamma(1 + 1/k)^2)
    # 解方程求解 k 和 lambda
    if seed is not None:
        np.random.seed(seed)
    def weibull_params(mean, std_dev):
        from scipy.optimize import fsolve
        def equations(vars):
            k, lambda_ = vars
            mean_eq = lambda_ * gamma(1 + 1/k) - mean
            std_dev_eq = lambda_ * np.sqrt(gamma(1 + 2/k) - gamma(1 + 1/k)**2) - std_dev
            return [mean_eq, std_dev_eq]
        
        k_guess, lambda_guess = 0.5, mean  # 初始猜测值
        k, lambda_ = fsolve(equations, (k_guess, lambda_guess))
        return k, lambda_

    k, lambda_ = weibull_params(mean, std_dev)
    samples = np.random.weibull(k, size=(time_horizon, M)) * lambda_    # 生成韦伯分布的随机样本
    samples = np.round(samples).astype(int)    # 将样本四舍五入为整数
    samples_df = pd.DataFrame(samples) # 转换为 DataFrame

    return samples_df

import numpy as np
import pandas as pd
from itertools import product

def get_leadtime(M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    # fix leadtime
    values_s1 = [1, 2, 3]
    prob_s1 = [0, 1, 0]  
    values_s2 = [1, 2, 3]
    prob_s2 = [0, 1, 0]
    values_s3 = [2, 3, 4]
    prob_s3 = [0, 1, 0]  

    all_combinations = np.array(list(product(values_s1, values_s2, values_s3)))
    valid_combinations = all_combinations[all_combinations.sum(axis=1) <= 6]

    selected_samples = np.zeros((8, M, 3), dtype=int)
    for i in range(8):
        for j in range(M):
            s1 = np.random.choice(values_s1, p=prob_s1)
            s2 = np.random.choice(values_s2, p=prob_s2)
            s3 = np.random.choice(values_s3, p=prob_s3)
                #if s1 + s2 + s3 <= 6:
            selected_samples[i, j, :] = [s1, s2, s3]


    result_dict = {
        f's{i+1}': pd.DataFrame(selected_samples[:, :, i],  # 取第 i 列
                                columns=range(M), 
                                index=pd.Index(range(8), name="time"))
        for i in range(3)
    }

    return result_dict

In [6]:
import numpy as np
import pandas as pd
import warnings
from scipy.stats import gamma

# ============================== Gaussian ==============================
def get_gaussian_demand(time_horizon, mean, std_dev, rho, M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    mean_vector = np.full((time_horizon,), mean)
    cov_value = rho * std_dev**2
    covariance_matrix = np.diag(np.full((time_horizon,), std_dev**2))

    for j in range(time_horizon - 1):
        covariance_matrix[j, j+1] = covariance_matrix[j+1, j] = cov_value

    warnings.filterwarnings("ignore", message="covariance is not symmetric positive-semidefinite")
    samples = np.random.multivariate_normal(mean_vector, covariance_matrix, M)
    samples = np.round(samples).astype(int)
    samples = np.clip(samples, 0, None)  # 设置负值为0
    samples_df = pd.DataFrame(samples.T)

    return samples_df

# ============================== Gamma ==============================
def get_gamma_demand(time_horizon, mean, std_dev, M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    k = mean**2 / std_dev**2
    theta = std_dev**2 / mean
    m = time_horizon * M
    samples = np.random.gamma(shape=k, scale=theta, size=m)
    samples = np.round(samples).astype(int)
    samples = np.clip(samples, 0, None)
    reshaped_array = samples.reshape((time_horizon, M))
    samples_df = pd.DataFrame(reshaped_array)

    return samples_df

# ============================== Lognormal ==============================
def get_lognormal_demand(time_horizon, mean, std_dev, rho, M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    normal_mean = np.log(mean**2 / np.sqrt(std_dev**2 + mean**2))
    normal_std_dev = np.sqrt(np.log(1 + (std_dev**2 / mean**2)))

    mean_vector = np.full((time_horizon,), normal_mean)
    cov_value = rho * normal_std_dev**2
    covariance_matrix = np.diag(np.full((time_horizon,), normal_std_dev**2))

    for j in range(time_horizon - 1):
        covariance_matrix[j, j+1] = covariance_matrix[j+1, j] = cov_value

    warnings.filterwarnings("ignore", message="covariance is not symmetric positive-semidefinite")
    samples = np.random.multivariate_normal(mean_vector, covariance_matrix, M)
    lognormal_samples = np.exp(samples)
    lognormal_samples = np.round(lognormal_samples).astype(int)
    lognormal_samples = np.clip(lognormal_samples, 0, None)
    samples_df = pd.DataFrame(lognormal_samples.T)

    return samples_df

# ============================== Weibull ==============================
from scipy.stats import gamma as gamma_dist      # 用于 Gamma 分布采样
from scipy.special import gamma as gamma_func    # 用于 Weibull 参数计算

def get_weibull_demand(time_horizon, mean, std_dev, M, seed=None):
    if seed is not None:
        np.random.seed(seed)

    from scipy.special import gamma as gamma_func
    from scipy.optimize import fsolve

    def weibull_params(mean, std_dev):
        def equations(vars):
            k, lambda_ = vars
            mean_eq = lambda_ * gamma_func(1 + 1/k) - mean
            std_dev_eq = lambda_ * np.sqrt(gamma_func(1 + 2/k) - gamma_func(1 + 1/k)**2) - std_dev
            return [mean_eq, std_dev_eq]
        return fsolve(equations, (0.5, mean))

    k, lambda_ = weibull_params(mean, std_dev)
    samples = np.random.weibull(k, size=(time_horizon, M)) * lambda_
    samples = np.round(samples).astype(int)
    samples = np.clip(samples, 0, None)
    samples_df = pd.DataFrame(samples)

    return samples_df

In [7]:
class Models:
    def __init__(self, h, b, I_0, B_0, R, input_parameters_file, dist, input_demand, N, input_price, input_leadtimes):
        
        # data_price = pd.read_excel(input_parameters_file, sheet_name = 'price')
        data_supplier = pd.read_excel(input_parameters_file, sheet_name = 'supplier')
        data_capacity = pd.read_excel(input_parameters_file, sheet_name = 'capacity')

        self.h = h
        self.b = b
        self.I_0 = I_0
        self.B_0 = B_0
        self.N = N
        # self.NP = NP
        self.Nlist = list(range(N))
        self.R = R
        self.dist = dist
        self.demand = input_demand
        self.price = input_price
        self.leadtime = input_leadtimes
        
        self.time = range(input_demand.shape[0])
        self.supplier, self.order_cost, self.lead_time, self.quality_level = self.get_suppliers(data_supplier)
        self.capacities = self.get_time_suppliers(data_capacity)
        
        self.t_n = self.get_structure(self.time, self.Nlist)
        # self.t_minus_n = self.get_structure(self.time[:-1], self.Nlist)
        self.t_supplier = self.get_structure(self.time, self.supplier)
        self.t_supplier_tprime = self.get_structure(self.time, self.supplier, self.time)
        self.t_n_m = self.get_structure(self.time, self.Nlist, list(range(N)))
        self.t_supplier_n = self.get_structure(self.time, self.supplier, list(range(N)))
        self.supplier_n = self.get_structure(self.supplier,self.Nlist)

    def get_structure(self, *args):
        if len(args) == 2:
            result = [(a, b) for a in args[0] for b in args[1]]
        elif len(args) == 3:
            result = [(a, b, c) for a in args[0] for b in args[1] for c in args[2]]
        elif len(args) == 4:
            result = [(a, b, c, d) for a in args[0] for b in args[1] for c in args[2]for d in args[3]]
            
        return result
   
    def get_suppliers(self, data_supplier):
        supplier = data_supplier['supplier'].values
        order_cost = data_supplier['order_cost'].values
        lead_time = data_supplier['lead_time'].values
        quality_level = data_supplier['quality_level'].values
        multi_temp = {}
        for i in range(len(supplier)):
            temp = [ order_cost[i], lead_time[i],quality_level[i]]
            multi_temp[supplier[i]] = temp
        return multidict(multi_temp)
    
    def get_time_suppliers(self, data_capacity):
        price_sn,  capacity_sn = [0], [0]
        for i in range(1, len(self.supplier)+1):
            # price_sn.append(data_price['s'+str(i)].values)
            capacity_sn.append(data_capacity['s'+str(i)].values)
        # prices = tupledict()
        capacities = tupledict()
        for t in self.time:
            for s in self.supplier:
                n = int(s[1:])
                # prices[(t,s)] = price_sn[n][t]
                capacities[(t,s)] = capacity_sn[n][t]
        return capacities 
    
    def display_data(self):
        print("Supplier Dictionary:", self.supplier)
        print("Order Cost Dictionary:", self.order_cost)
        print("Lead Time:", self.lead_time)
        print("Quality Level Dictionary:", self.quality_level)
        print("Prices Dictionary:", self.prices)
        print("Capacities Dictionary:", self.capacities)
        print("Input_demand:", self.demand)     
        
########################==============================models for DRO Price==============================########################   
    def optimize_DROprice (self,epsilon):

        time_n = self.get_structure(self.time, self.Nlist)
        time_supplier_n = self.get_structure(self.time, self.supplier, self.Nlist)
        

        P_I = self.I_0 - self.B_0
                                                               
        model = gp.Model('DROprice')

        Q = model.addVars(self.t_supplier,lb=0,vtype=gp.GRB.CONTINUOUS, name = 'order_quantity')
        theta = model.addVars(self.t_supplier,lb=0, vtype=gp.GRB.CONTINUOUS,name="arrive_quantity")
        Y = model.addVars(self.t_supplier,lb=0, vtype=gp.GRB.BINARY,name="if_make_order_arrive")
        I = model.addVars(self.time, lb=0, vtype=gp.GRB.CONTINUOUS, name = 'inventory')
        B = model.addVars(self.time, lb=0, vtype=gp.GRB.CONTINUOUS, name = 'backlog')
        beta = model.addVars(self.supplier, lb=0, vtype=gp.GRB.CONTINUOUS, name = 'beta')
        alpha = model.addVars(self.supplier_n, vtype=gp.GRB.CONTINUOUS, name = 'alpha')

        xi1 = model.addVars(self.t_supplier_n, lb=0, vtype=gp.GRB.CONTINUOUS)
        xi2 = model.addVars(self.t_supplier_n, lb=0, vtype=gp.GRB.CONTINUOUS)
        xi3 = model.addVars(self.supplier_n, lb=0, vtype=gp.GRB.CONTINUOUS)
        
        model.update()
        model.setParam('Outputflag',0)
        
        obj0 = 0
        for n in self.Nlist:
            obj0 += quicksum(self.h * I[t] + self.b * B[t] for t in self.time)
        obj1 = obj0 / self.N + quicksum(self.order_cost[s] * Y[t,s]  for t in self.time for s in self.supplier)
        obj1 += quicksum(beta[s]*epsilon[s] + quicksum(alpha[s,n] for n in self.Nlist) / self.N for s in self.supplier)
        
        model.setObjective(obj1, GRB.MINIMIZE)

        for t in self.time:
            if t == 0:
                model.addConstr(I[t] - B[t] == P_I + sum(theta[t,s] for s in self.supplier) - 1807, name=f"inv_balance_{t}")
            else:
                model.addConstr(I[t] - B[t] == I[t - 1] - B[t - 1] + sum(theta[t,s] for s in self.supplier) - 1807, name=f"inv_balance_{t}")
                               
        for s in self.supplier:
            for t_prime in self.time:
                for n in self.Nlist:
                    model.addConstr(theta[t_prime,s] == sum(Q[t, s] for t in self.time if t + self.leadtime[s].iloc[t,n] == t_prime),name=f"theta_cal_{t_prime}_{s}")  
            
        for s in self.supplier:
            model.addConstrs(xi3[s,n] <= beta[s] for n in self.Nlist)
            model.addConstrs(quicksum(self.price[s].iloc[t,n]*(xi1[t,s,n] -xi2[t,s,n]) for t in self.time) <= alpha[s,n] for n in self.Nlist)
            for t in self.time:
                model.addConstr(Q[t,s] <=  Y[t,s] * 999999 )
                model.addConstr(Q[t,s] <= self.capacities[t,s])
                model.addConstrs(Q[t,s] -xi1[t,s,n] +xi2[t,s,n] <= 0 for n in self.Nlist)
                model.addConstrs(xi1[t,s,n] +xi2[t,s,n] -xi3[s,n] <= 0 for n in self.Nlist)

        model.optimize()

        variables = model.getVars()
        df_result = pd.DataFrame([(var.VarName, var.X) for var in variables],
                      columns=['variable_name', 'value'])

        return model.objVal, df_result
    
########################==============================models for SAA==============================########################   
    def optimize_SAA (self):

        time_n = self.get_structure(self.time, self.Nlist)
        time_supplier_n = self.get_structure(self.time, self.supplier, self.Nlist)
        t_supplier_tprime_n = self.get_structure(self.time, self.supplier, self.time, self.Nlist)
        P_I = self.I_0 - self.B_0
                                                               
        model = gp.Model('SAA')

        Q = model.addVars(self.t_supplier,lb=0,vtype=gp.GRB.CONTINUOUS, name = 'order_quantity')
        theta = model.addVars(self.t_supplier_n,lb=0, vtype=gp.GRB.CONTINUOUS,name="arrive_quantity")
        Y = model.addVars(self.t_supplier,lb=0, vtype=gp.GRB.BINARY,name="if_make_order_arrive")
        I = model.addVars(self.t_n, lb=0, vtype=gp.GRB.CONTINUOUS, name = 'inventory')
        B = model.addVars(self.t_n, lb=0, vtype=gp.GRB.CONTINUOUS, name = 'backlog')
        
        model.update()
        model.setParam('Outputflag',0)
        
        obj0 = 0
        for n in self.Nlist:
            obj0 += quicksum(self.price[s].iloc[t,n] * Q[t,s] for t in self.time for s in self.supplier)
            obj0 += quicksum(self.h * I[t,n] + self.b * B[t,n] for t in self.time)
        obj1 = obj0 / self.N
        obj2 = quicksum(self.order_cost[s] * Y[t,s]  for t in self.time for s in self.supplier)
        
        model.setObjective(obj1 + obj2, GRB.MINIMIZE)

        for n in self.Nlist:
            for t in self.time:
                if t == 0:
                    model.addConstr(I[t,n] - B[t,n] == P_I + sum(theta[t,s,n] for s in self.supplier) - self.demand.iloc[t,n], name=f"inv_balance_{t}")
                else:
                    model.addConstr(I[t,n] - B[t,n] == I[t - 1, n] - B[t - 1, n] + sum(theta[t,s,n] for s in self.supplier) - self.demand.iloc[t,n], name=f"inv_balance_{t}")
                               
        # print(self.lead_time)
        for s in self.supplier:
            for t_prime in self.time:
                for n in self.Nlist:
                    model.addConstr(theta[t_prime,s,n] == sum(Q[t, s] for t in self.time if t + self.leadtime[s].iloc[t,n] == t_prime),name=f"theta_cal_{t_prime}_{s}")
        
        for n in self.Nlist:              
            model.addConstrs(Q.sum(t, s) <=  Y[t,s] * 999999 for t,s in self.t_supplier)
            
        model.addConstrs( Q[t, s] <= self.capacities[t,s] for t,s in self.t_supplier) 

        model.optimize()

        variables = model.getVars()
        df_result = pd.DataFrame([(var.VarName, var.X) for var in variables],
                      columns=['variable_name', 'value'])

        return model.objVal, df_result

## same demand and lead time, diff price sample

In [ ]:
input_parameters_file = 'input_parameters.xlsx'
input_demand_file = 'input_demand_same.xlsx'
input_leadtime_file = 'input_leadtimes_same.xlsx'

# input price sample from .xlsx
input_price_file = 'input_prices_diff.xlsx'
#input_price_file = 'input_prices_diff larger std.xlsx'    # if you check the case with a larger std,  you need to reset: input_price_std = {'s1':6, 's2':5, 's3':5} in line 24

h = 5 # inventory holding cost
b = 30   # backlog cost
I_0 = 1800    # starting inventory level
B_0 = 0    
R = 0

k_fold = 5
oos_size = 10000
planning_horizon = 8
rho = 0 # co-variance in demand

input_demand_mean = 1807
#input_demand_std = 488
input_demand_std = 0.01    # fix demand equal to mean = 1807
input_price_mean = {'s1':22, 's2':23, 's3':21.40}
input_price_std = {'s1':4, 's2':3, 's3':4}  
seed = 25

# since fixed demand, demand is independent of distribution
oos_demands = get_gaussian_demand(planning_horizon, input_demand_mean, input_demand_std, rho, oos_size, seed)

oos_prices_normal, oos_prices_gamma, oos_prices_lognormal, oos_prices_weibull = {}, {}, {}, {}
oos_leadtimes = get_leadtime(oos_size)
for s in input_price_std:
    mean, std = input_price_mean[s], input_price_std[s]

    one_price_normal = get_gaussian_demand(planning_horizon, mean, std, rho, oos_size, seed)
    oos_prices_normal[s] = one_price_normal

    one_price_gamma = get_gamma_demand(planning_horizon, mean, std, oos_size, seed)
    oos_prices_gamma[s] = one_price_gamma

    one_price_lognormal = get_lognormal_demand(planning_horizon, mean, std, rho, oos_size, seed)
    oos_prices_lognormal[s] = one_price_lognormal
    
    one_price_weibull = get_weibull_demand(planning_horizon, mean, std, oos_size, seed)
    oos_prices_weibull[s] = one_price_weibull
    
oos_prices_dict = {'normal':oos_prices_normal, 'gamma':oos_prices_gamma, 'log_normal':oos_prices_lognormal, 'weibull': oos_prices_weibull}

In [10]:
input_sample_no = [5,10,20,40,60,80,100,120,140,160,180,200][:]
oos_analys = OOS_analys(h, b, I_0, B_0, input_parameters_file)
all_res = {}
all_df = []
for input_dist in ['normal','gamma','log_normal', 'weibull'][:]:
    # all_res['demand_uncertainty'] = input_demand_file.split('_')[-1].split('.')[0]
    # out_sample_dist = input_dist
    all_res['input_dist'] = input_dist
    
    for out_sample_dist in ['normal','gamma','log_normal', 'weibull'][:]:
        all_res['oos_dist'] = out_sample_dist

        #oos_demands = oos_demands_dict[out_sample_dist]

        for N in input_sample_no:
            all_res['N'] = N
            input_demand = pd.read_excel(input_demand_file, sheet_name = input_dist, index_col=0, usecols=range(N+1))   #choose the first N demands
            #for mark in ['diff']:
            
            input_prices = {}# oos_prices_dict[out_sample_dist]
            input_leadtimes = {}
                    
            for s in input_price_std:
                input_prices[s] = pd.read_excel(input_price_file, sheet_name = s+input_dist, index_col=0, usecols=range(N+1), engine='openpyxl' )#, usecols=range(Np+1))
                #input_prices[s] = input_prices_dict[input_dist][s]
                input_leadtimes[s] = pd.read_excel(input_leadtime_file, sheet_name = s, index_col=0, usecols=range(N+1), engine='openpyxl' )
########-------------------------------Cross validation--------------------------------
            cross_res = {}
            cross_res['SAA'] = -1
            for model_name in ['DROprice']:
                min_epsilons = [0] * k_fold
                for k in range(k_fold):
                    # use the \k-th sample as the training sample
                    num_fold = N // k_fold
                    train_size = N - (N // k_fold)
                    all_columns = list(range(input_demand.shape[1]))
                    selected_columns = [i for i in all_columns if i < k*num_fold or i >= (k+1)*num_fold]
                    train_demand = input_demand.iloc[:, selected_columns]
                        # print("train_demand: \n", train_demand)
                    train_price = {}
                    train_leadtime={}
                    for s in input_price_std:
                        train_price[s] = input_prices[s].iloc[:, selected_columns]
                        train_leadtime[s] = input_leadtimes[s].iloc[:, selected_columns]
                        # print("train_price: \n", train_price)
                    solve = Models(h, b, I_0, B_0, R, input_parameters_file, input_dist, train_demand, train_size, train_price,train_leadtime)
                    
                    min_cost = 1e11
                    list_epsilon = [0,1,5,10]
                    min_epsilons[k] = {}
                    eps = {}
                    for eps['s1'] in list_epsilon:
                        for eps['s2'] in list_epsilon:
                            for eps['s3'] in list_epsilon:
                                CV_input_prices = {}
                                obj, solution = solve.optimize_DROprice(eps)

                                # cost, _ = oos_analys.cal_out_of_sample(solution, cross_validation_demands)# use the oos samples
                                CV_input_demand = input_demand.iloc[:, k*num_fold : (k+1)*num_fold]
                                CV_input_demand.columns = range(num_fold)
                                CV_input_price = {}
                                CV_input_leadtime = {}
                                for s in input_price_std:
                                    CV_input_price[s] = input_prices[s].iloc[:, k*num_fold : (k+1)*num_fold]
                                    CV_input_price[s].columns = range(num_fold)
                                    CV_input_leadtime[s] = input_leadtimes[s].iloc[:, k*num_fold : (k+1)*num_fold]
                                    CV_input_leadtime[s].columns = range(num_fold)
                
                                    # print(CV_input_demand)
                                cost, _ = oos_analys.cal_out_of_sample(solution, CV_input_demand, CV_input_price, CV_input_leadtime)# use the k-th samples
                                #print(cost.total_cost)
                                tmp = cost.total_cost.mean()
                                
                                if tmp < min_cost:
                                    min_cost = tmp
                                    for s in input_price_std:
                                        min_epsilons[k][s] = eps[s]
                                    #print(min_epsilons)
                                    
                keys = min_epsilons[0].keys()
                cross_res[model_name] = {k: sum(d[k] for d in min_epsilons) / len(min_epsilons) for k in keys}
                #cross_res['McC'] = 10
                #cross_res['CPP'] = 50
            #cross_res={'DROprice': {'s1': 0, 's2':0, 's3': 0}}
            print(cross_res)
            for model_name in ['DROprice','SAA']:
                print('input_dist=', input_dist,', sample sizes=', N,', out_sample_dist=',out_sample_dist, model_name)
                start = time.time()
                solve = Models(h, b, I_0, B_0, R, input_parameters_file, input_dist, input_demand, N, input_prices, input_leadtimes)
                if model_name == 'DROprice':
                    obj, solution = solve.optimize_DROprice(cross_res[model_name])
                if model_name == 'SAA':
                    obj, solution = solve.optimize_SAA()
                        
                oos_cost, oos_details = oos_analys.cal_out_of_sample(solution, oos_demands, oos_prices_dict[out_sample_dist], oos_leadtimes)
    
                key = [input_dist, N, model_name, out_sample_dist, obj,'DRO']
                #oos_analys.plot_solution(key,solution)
                print('oos mean', oos_cost.total_cost.mean())
                end = time.time()
                print((end - start) / 60)
    
                all_res['model_name'] = model_name
                all_res['obj'] = obj
                all_res['epsilon'] = cross_res[model_name]
                all_res['oos_mean'] = oos_cost.total_cost.mean()
                all_res['order']= oos_cost.fixed_order_cost.mean()
                all_res['purchase']= oos_cost.purchase_cost.mean()
                all_res['oos_inv'] = oos_cost.inv_cost.mean()
                all_res['oos_backlog'] = oos_cost.backlog_cost.mean()
                all_res['oos_std'] = oos_cost.total_cost.std()
                all_res['time_min'] = (end - start) / 60
                all_df.append(pd.DataFrame.from_dict(all_res, orient='index'))

{'SAA': -1, 'DROprice': {'s1': 4.0, 's2': 1.0, 's3': 6.0}}
input_dist= normal , sample sizes= 5 , out_sample_dist= normal DROprice
oos mean 334345.0235
1.6711233576138815
input_dist= normal , sample sizes= 5 , out_sample_dist= normal SAA
oos mean 331630.9984
1.6992462356885274
{'SAA': -1, 'DROprice': {'s1': 5.0, 's2': 0.2, 's3': 4.0}}
input_dist= normal , sample sizes= 10 , out_sample_dist= normal DROprice
oos mean 333336.9483
1.6323588132858275
input_dist= normal , sample sizes= 10 , out_sample_dist= normal SAA
oos mean 331716.1329
1.5267493804295857


# Due to some parameter adjustments, it may not yield the same results as in the slice, but DRO is still inferior to SAA

In [97]:
# input_price_mean = {'s1':22, 's2':23, 's3':21.4}
# input_price_std = {'s1':10, 's2':10, 's3':10}

pd.options.display.float_format = '{:.2f}'.format
pd.concat(all_df, axis=1).T

,input_dist,oos_dist,N,model_name,obj,epsilon,oos_mean,order,purchase,oos_inv,oos_backlog,oos_std,time_min
0,normal,log_normal,80,DROprice,347472.86,"{'s1': 2.0, 's2': 0.0, 's3': 3.0}",340637.24,70.00,285937.24,0.00,54630.00,55688.13,0.03
0,normal,log_normal,80,SAA,338691.19,-1,338251.51,80.00,283541.51,0.00,54630.00,69633.76,0.03
0,gamma,log_normal,80,DROprice,332749.94,"{'s1': 1.0, 's2': 0.0, 's3': 5.0}",340637.24,70.00,285937.24,0.00,54630.00,55688.13,0.03
0,gamma,log_normal,80,SAA,324949.12,-1,338451.03,90.00,283731.03,0.00,54630.00,64222.23,0.03
0,log_normal,log_normal,80,DROprice,343766.50,"{'s1': 1.0, 's2': 0.0, 's3': 3.0}",339598.22,70.00,284898.22,0.00,54630.00,72664.86,0.03
0,log_normal,log_normal,80,SAA,336269.40,-1,339319.46,70.00,284619.46,0.00,54630.00,72656.42,0.04
0,weibull,log_normal,80,DROprice,327207.86,"{'s1': 0.0, 's2': 0.0, 's3': 3.0}",338692.80,110.00,283952.80,0.00,54630.00,69294.97,0.04
0,weibull,log_normal,80,SAA,321894.00,-1,338479.95,80.00,283769.95,0.00,54630.00,70234.88,0.03


In [87]:
# input_price_std = {'s1':6, 's2':5, 's3':5}
pd.options.display.float_format = '{:.2f}'.format
pd.concat(all_df, axis=1).T

,input_dist,oos_dist,N,obj,epsilon,oos_mean,order,purchase,oos_inv,oos_backlog,oos_std,time_min
0,normal,normal,80,344090.84,"{'s1': 4.0, 's2': 0.0, 's3': 2.0}",336284.34,100.00,281554.34,0.00,54630.00,35829.96,0.04
0,normal,normal,80,335292.16,-1,335647.56,90.00,280927.56,0.00,54630.00,39300.23,0.04
0,gamma,normal,80,334449.00,"{'s1': 2.0, 's2': 0.0, 's3': 2.0}",337100.71,120.00,282350.71,0.00,54630.00,32524.36,0.03
0,gamma,normal,80,327420.71,-1,336984.74,90.00,282264.74,0.00,54630.00,35701.41,0.03
0,log_normal,normal,80,342198.75,"{'s1': 3.0, 's2': 0.0, 's3': 2.0}",336284.34,100.00,281554.34,0.00,54630.00,35829.96,0.03
0,log_normal,normal,80,334395.38,-1,335647.56,90.00,280927.56,0.00,54630.00,39300.23,0.03
0,weibull,normal,80,325187.88,"{'s1': 0.0, 's2': 0.0, 's3': 0.0}",336364.65,80.00,281654.65,0.00,54630.00,38334.47,0.04
0,weibull,normal,80,325187.88,-1,336364.65,80.00,281654.65,0.00,54630.00,38334.47,0.03
